# Lab Exercise 1

Exercise description in German.

## Klassisches Kalman-Filter

### Problemstellung

Aktienkurse unterliegen einem mehr oder wenig starken Rauschen. Beispielhaft ist der Aktienkurs des Unternehmens „Leoni“ dargestellt.


In [ ]:
import pandas as pd

leoni_data = pd.read_csv(
    "data/Leoni.dat",
    sep=";",
    parse_dates=["Datum"],
    date_format=r"%d.%m.%Y",
    index_col="Datum",
)
leoni_data

In [ ]:
leoni_data.plot(
    y="Schlusskurs",
    kind="line",
    grid=True,
    ylabel="Aktienkurs [€]",
    title="Eingangsdaten",
)

### Aufgabenstellung

Entwerfen Sie ein Kalman-Filter (Beta-Filter) um das Rauschen im Aktienkurs zu minimieren. 

Hinweis zur Bearbeitung: Bestimmen Sie zuerst für den Zeitraum die Varianz $R$ des Messrauchens. Gehen Sie davon aus, dass sich die Varianz des Rauschens zeitinvariant ist.

Variieren Sie dann das Systemrauchen $Q$ und schauen sich den Verlauf der beiden Zustandsgrößen an.

Optimieren Sie diesen Kalman-Filter so, dass er eine minimale Laufzeit besitzt.

In [ ]:
import matplotlib.pyplot as plt
import numpy as np

In [ ]:
y = leoni_data["Schlusskurs"].to_numpy()
u = np.zeros((y.size, 1))
t = leoni_data["Eintrag"].to_numpy()

Ts = t[1] - t[0]
R = y.var()
Q = 100

Ad = np.array([[1, Ts], [0, 1]])
Bd = np.array([[0], [0]])
C = np.array([[1, 0]])
D = 0
Gd = np.array([[Ts], [1]])

In [ ]:
x_hat = np.array([[y[0]], [0]])
P_hat = 50 * np.eye(2)

d_y = np.zeros_like(y)

s = np.zeros_like(y)
v = np.zeros_like(y)
K1 = np.zeros_like(y)
K2 = np.zeros_like(y)
P_tilde1 = np.zeros_like(y)
P_tilde2 = np.zeros_like(y)
P_tilde3 = np.zeros_like(y)
P_tilde4 = np.zeros_like(y)
for i, y_i in enumerate(y):
    d_y[i] = y_i - (C @ x_hat + D * u[i])
    k = C @ P_hat @ C.T + R
    if k.ndim >= 2:
        K = P_hat @ C.T @ np.linalg.pinv(C @ P_hat @ C.T + R)
    else:
        K = P_hat @ C.T * 1 / (C @ P_hat @ C.T + R)
    x_tilde = x_hat + (K * d_y[i]).reshape(-1, 1)
    P_tilde = (np.eye(Bd.size) - K @ C) @ P_hat

    x_hat = Ad @ x_tilde + (Bd @ u[i]).reshape(-1, 1)
    P_hat = Ad @ P_tilde @ Ad.T + (Gd * Q) @ Gd.T

    s[i], v[i] = x_tilde
    K1[i], K2[i] = K
    P_tilde1[i], P_tilde2[i] = P_tilde[0, :]
    P_tilde3[i], P_tilde4[i] = P_tilde[1, :]

fig, (ax_s, ax_v) = plt.subplots(2, 1, figsize=(15, 7))

ax_s.plot(t, y, "k*")
ax_s.plot(t, s, "b--")
ax_s.plot(t, s + 1.5 * np.sqrt(P_tilde1), "b-")
ax_s.plot(t, s - 1.5 * np.sqrt(P_tilde1), "b-")
ax_s.grid(True)
ax_s.set_xlabel("Zeit [Tag]")
ax_s.set_ylabel("Kurs [€]")

ax_v.plot(t, v, "r-*")
ax_v.grid(True)
ax_v.set_xlabel("Zeit [Tag]")
ax_v.set_ylabel("Kursänderung [€/Tag]")

fig.tight_layout()